In [2]:
from roblox_ml.data import DataConnector
import numpy as np
import json
import datasets
import os
import shutil

/home/jovyan/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
universe_detail_query = """
    SELECT
        id AS universe_id,
        name AS name,
        description AS description
    FROM sqlserver.robloxuniverses_universes WHERE 
        """

bdir = '/home/jovyan/data/nhirschkind/icon_sample/train'
fnames = os.listdir(bdir)
uids = [int(eval(x.split('.')[0])) for x in fnames]
for uid in uids[:-1]:
    universe_detail_query += "ds='2023-12-01' AND id=" + str(uid) + ' OR\n'
universe_detail_query += "ds='2023-12-01' AND id=" + str(uids[-1])

dc = DataConnector()
result = dc.hive_query_to_dataframe(universe_detail_query)

/tmp/ipykernel_915/4185519867.py:17: DeprecatedWarning: hive_query_to_dataframe is deprecated. Use the "sparkconnector" class instead
  result = dc.hive_query_to_dataframe(universe_detail_query)


In [15]:
result.head()

,universe_id,name,description
0,2305124262,Water Park World [Beta],Water Park World is a tycoon game in which you...
1,2308385837,Panem | District 12,"In order to gain access to District Twelve, yo..."
2,2308525745,Mr. Sti t c hy! Piggy Jumpscare Test,"Hello !, This Game Is A FanGame Of The Famous ..."
3,2277158287,Cruise Ship Sinking,She Hit a Rock And Sunk\r\nCan You Survive Her...
4,2257350744,백제 풍납토성,현재 개발 중 입니다.\r\n맵은 개발이 완료 되었고 현재 복장이나 스크립트 작성으...


In [26]:
dictform = {}
for i, row in result.iterrows():
    dictform[row['universe_id']] = (row['name'], row['description'])

prompt_template = 'The thumbnail of a Roblox game called "{name}", described by its creator as "{description}"'
metadata = []
for uid in uids:
    name, description = dictform[uid]
    if not isinstance(name, str):
        continue
    if description is None:
        description = "(No description available)"
    #name = name.replace('\n',' ')
    #description = description.replace('\n', ' ')
    prompt = prompt_template.format(name=name, description=description)
    metadata.append({'file_name': str(uid) + '.png', 'text': prompt})


In [27]:
import json
with open('/home/jovyan/data/nhirschkind/icon_sample/train/metadata.jsonl', 'w') as f:
    for entry in metadata:
        json.dump(entry, f)
        f.write('\n')